In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "wiki-research-l/2020-July"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [28]:
len(mail_df)

37

In [4]:
from preProcess import cleaningText
s_list = []
for i,content in mail_df['Content'].items():
    sentences = cleaningText(content)
    s_list.append(sentences)
mail_df['Cleaned_Content'] = s_list

In [5]:
mail_df.head()

,file_path,Message-ID,From,Date,Subject,Content,Cleaned_Content
0,wiki-research-l/2020-July/2020-July-30,<CAHTOhHs0k6q89WAHek45sA9ePjLqYhYEEq0pu6GOdX1pqBL1Tg@mail.gmail.com>,haifengz.suny at gmail.com (Haifeng Zhang),"Wed, 22 Jul 2020 10:23:48 -0400",[Wiki-research-l] Qid to Page ID,"Paul,\nSorry, I did not read your response carefully.\nWill try the wikimapper API as suggested.\n\nThanks a lot!\n\nOn Tue, Jul 21, 2020 at 1:05 PM Paul Albert <mrpaulalbert at gmail.com> wrote:\n\n>\n> I've been using the Wikimapper API mentioned here:\n>\n>\n> https://opendata.stackexchange.com/questions/5467/mapping-all-wikipedia-urls-to-wikidata-entity-ids\n>\n>\n> Regards,\n>\n> Paul Albert\n>\n> ﻿On 7/21/20, 12:39 PM, ""Wiki-research-l on behalf of Haifeng Zhang"" <\n> wiki-research-l-bounces at lists.wikimedia.org on behalf of\n> haifengz.suny at gmail.com> wrote:\n>\n> Hi all,\n> Are you aware of any API that maps wikidata QIDs to their article Page\n> IDs?\n>\n> Thanks,\n> Haifeng\n> _______________________________________________\n> Wiki-research-l mailing list\n> Wiki-research-l at lists.wikimedia.org\n> https://lists.wikimedia.org/mailman/listinfo/wiki-research-l\n> _______________________________________________\n> Wiki-research-l mailing...","Paul, Sorry, I did not read your response carefully. Will try the wikimapper API as suggested. Thanks a lot!"
1,wiki-research-l/2020-July/2020-July-08,<CAJXKj+owEP_UyX2A17x+4NrsB76zv9puwpbPE8ZPBpfD0CRARQ@mail.gmail.com>,jlayton at wikimedia.org (Janna Layton),"Thu, 9 Jul 2020 12:26:49 -0700","[Wiki-research-l] [Wikimedia Research Showcase] July 15,\n\t2020: Medical Knowledge on Wikipedia","Hi all,\n\nThe next Research Showcase will be live-streamed on Wednesday, July 15, at\n9:30 AM PDT/16:30 UTC.\n\nWikipedia is one of the most important online resources for health\ninformation. This has been especially highlighted during the Covid-19\npandemic: since the beginning of the year more than 5000 articles related\nto Covid-19 have been created receiving more than 400M pageviews.\nTherefore, for this month’s showcase our two invited speakers will help us\nget a better understanding of the state of medical knowledge in Wikipedia.\nIn the first talk, Denise Smith will give an overview on how Wikipedia's\nhealth content is used by different audiences (public, students, or\npractitioners). In the second talk, Giovanni Colavizza will present results\non how editors on Wikipedia find, select, and integrate scientific\ninformation on Covid-19 into Wikipedia articles.\n\nYouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c\n\nAs usual, you can join the conversation on IRC ...","Hi all, The next Research Showcase will be live-streamed on Wednesday, July 15, at 9:30 AM PDT/16:30 UTC. Wikipedia is one of the most important online resources for health information. This has been especially highlighted during the Covid-19 pandemic: since the beginning of the year more than 5000 articles related to Covid-19 have been created receiving more than 400M pageviews. Therefore, for this month’s showcase our two invited speakers will help us get a better understanding of the state of medical knowledge in Wikipedia. In the first talk, Denise Smith will give an overview on how Wikipedia's health content is used by different audiences (public, students, or practitioners). In the second talk, Giovanni Colavizza will present results on how editors on Wikipedia find, select, and integrate scientific information on Covid-19 into Wikipedia articles. YouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c As usual, you can join the conversation on IRC at #wikimedia-research...."
2,wiki-research-l/2020-July/2020-July-01,<CAE4fJj_b7k8yeqmz19a-seo3gtL9Mg1nvQO0oPUtrxvCFMkEaw@mail.gmail.com>,ziningy1 at andrew.cmu.edu (Ethan Ye),"Wed, 1 Jul 2020 17:54:24 -0400",[Wiki-research-l] Wikipedia ORES Research Study Looking for\n\tParticipants,"Hi Stella,\n\nThanks for clarifying. So unfortunately in this research, we are only\nlook

In [16]:
#　センテンスのリストを作る
sentence_list = []
for content in mail_df['Cleaned_Content']:
    sentences = tokenize.sent_tokenize(content)
    for sentence in sentences:
        sentence_list.append(sentence)

In [24]:
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [75]:
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
        rows.append(row)

In [76]:
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])

In [80]:
from db import connect
engine = connect()
kb_df.to_sql(name='kb-wiki-reseach-l',con=engine,if_exists='replace',index=None)